# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** - проверить четыре гипотезы:
1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

## Шаг 1. Обзор данных

Составим первое впечатление о статистике платёжеспособности клиентов, полученной от банка.

Импортируем библиотеку `pandas` и `pymystem3`:

In [1]:
# импорт библиотеки pandas
import pandas as pd

# импортируем pymystem3
from pymystem3 import Mystem
m = Mystem()

Прочитаем файл `data.csv` из папки `/datasets` и сохраним его в переменной `data`, выведем на экран первые 10 строк таблицы:

In [2]:
# чтение файла с данными и сохранение в data
data = pd.read_csv('/datasets/data.csv')

# получение первых 10 строк таблицы data
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Нельзя не заметить некорректное представление информации о трудовом стаже в столбце `days_employed`. Возможно причина в проблемах с выгрузкой данных. Так как трудовой стаж не влияет на результат выдвинутых гипотез, в дальнейшем его предлагается **исключить из рассмотрения**.

Получим общую информацию о таблице:

In [3]:
# получение общей информации о данных в таблице
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В названиях столбцов нет расхождений с хорошим стилем:

In [4]:
# перечень названий столбцов таблицы
data.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

Итак: 
- В таблице **12 столбцов**. 
- В двух столбцах есть пропущенные значения. 
- В столбце с доходами неудобный формат данных.

Из документации мы знаем описание каждого столбца с данными, а тип данных, хранящихся в столбцах, определили с помощью функции `info()`:
* `children` — количество детей в семье, тип данных `int64`
* `days_employed` — общий трудовой стаж в днях, тип данных `float64`
* `dob_years` — возраст клиента в годах, тип данных `int64`
* `education` — уровень образования клиента, тип данных `object`
* `education_id` — идентификатор уровня образования, тип данных `int64`
* `family_status` — семейное положение, тип данных `object`
* `family_status_id` — идентификатор семейного положения, тип данных `int64`
* `gender` — пол клиента, тип данных `object`
* `income_type` — тип занятости, тип данных `object`
* `debt` — имел ли задолженность по возврату кредитов, тип данных `int64`
* `total_income` — ежемесячный доход, тип данных `float64`
* `purpose` — цель получения кредита, тип данных `object`

Итого: 
* 2 столбца с типом данных `float64`
* 5 столбцов с типом данных  `int64`
* 5 столбцов с типом данных `object`

**Вывод**

В каждой строке таблицы — данные о заёмщике и его кредите. 

Часть колонок описывает заёмщика: количество детей, общий трудовой стаж, возраст, уровень образования, семейное положение, пол, тип занятости, ежемесячный доход. 

Остальные данные рассказывают о кредите заёмщика:  есть ли задолженность по кредиту, какая была цель получения кредита. 

Некоторые столбцы не потребуются для проведения исследования, так как не имеют отношения к поставленным гипотезам. 

Предварительно можно утверждать, что данных достаточно для проверки гипотез, но встречаются пропуски в данных.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Шаг 2. Предобработка данных

### Выбор информации для анализа, разведка данных

Для проверки выдвинутых гипотез в исследовании не потребуются некоторые исходные данные. Учитывая, что нас интересует:
- информация о наличии детей, 
- семейном положении, 
- уровне дохода и целях кредита, 
- а также информация о задолженности по кредиту, 

оставим для рассмотрения следующие столбцы:

* `children` — количество детей в семье, тип данных `int64`
* `family_status` — семейное положение, тип данных `object`
* `family_status_id` — идентификатор семейного положения, тип данных `int64`
* `total_income` — ежемесячный доход, тип данных `float64`
* `purpose` — цель получения кредита, тип данных `object`
* `debt` — имел ли задолженность по возврату кредитов, тип данных `int64`

Но **важно** *обработать явные дубликаты* **до выделения столбцов с нужной информацией**. Так как при исключении из рассмотрения возраста и другой информации, которая не рассматривается для проверки гипотез, количество явных дубликатов может увеличиться, хотя рассматриваться будут разные люди.

Количество явных дубликатов до предобработки исходных данных:

In [5]:
#явные дубликаты в основной исходной таблице
data.duplicated().sum()

54

Чтобы более-менее аккуратно удалить явные дубликаты проведём небольшую подготовку данных в столбцах, которые нам не потребуются в дальнейшем. Начнём со столбца `education`, посмотрим какие значения в нём встречаются:

In [6]:
#просмотр уникальных значений столбца 'education'
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Переведём названия уровня образования клиентов в нижний регистр, чтобы исключить неявные дубликаты:

In [7]:
#перевод значений столбца 'education' в нижний регистр
data['education'] = data['education'].str.lower()

Проверим результат операции:

In [8]:
#просмотр уникальных значений столбца 'education'
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Теперь посмотрим идентификаторы уровня образования, здесь всё в порядке, явных дубликатов нет, количество соответствует названиям уровня образования:

In [9]:
#просмотр уникальных значений столбца 'education_id'
data['education_id'].value_counts()

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

Проверим есть ли дубликаты в парах идентификатор-уровень образования клиентов. Для этого создадим словарь уровня образования клиентов:

In [10]:
#создание словаря уровня образования
education_dict = data[['education_id','education']]

Выведем на экран несколько значений:

In [11]:
# вывод на экран первых 10 строк таблицы
education_dict.head(10)

,education_id,education
0,0,высшее
1,1,среднее
2,1,среднее
3,1,среднее
4,1,среднее
5,0,высшее
6,0,высшее
7,1,среднее
8,0,высшее
9,1,среднее


В словаре образования клиентов неявных дубликатов не обнаружено:

In [12]:
#удаление дубликатов для проверки соответствия одного значения 
#"education_id" одному уровню образования
education_dict.drop_duplicates().reset_index(drop=True)

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


После изменения регистра в названиях уровня образования клиентов в столбце `education` увеличилось количество явных дубликатов в основной таблице исходных данных:

In [13]:
#явные дубликаты в основной таблице исходных данных
data.duplicated().sum()

71

Далее рассмотрим столбец `gender`.

In [14]:
#просмотр уникальных значений столбца 'gender'
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Неявных дубликатов не обнаружено, неопознанный пол 'XNA' изменять не будем, так как это не повлияет на результаты.

Далее рассмотрим столбец `income_type`.

In [15]:
#просмотр уникальных значений столбца 'income_type'
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

Неявных дубликатов не обнаружено. 

В результате анализа столбцов основной таблицы **можно удалять явные дубликаты**:

In [16]:
# удаление явных дубликатов (с удалением старых индексов и формированием новых)
data = data.drop_duplicates().reset_index(drop=True) 

Ещё раз посчитаем явные дубликаты в таблице — убедимся, что полностью от них избавились:

In [17]:
#проверка отсутствия явных дубликатов
data.duplicated().sum()

0

После удаления явных дубликатов в основной таблице исходных данных **перейдём к выделению нужной для исследования информации**:

In [18]:
# создаём новую таблицу с нужными для исследования столбцами, исходную сохраняем
df = data[['children','family_status','family_status_id','total_income','purpose','debt']]
df.head(10)

,children,family_status,family_status_id,total_income,purpose,debt
0,1,женат / замужем,0,253875.639453,покупка жилья,0
1,1,женат / замужем,0,112080.014102,приобретение автомобиля,0
2,0,женат / замужем,0,145885.952297,покупка жилья,0
3,3,женат / замужем,0,267628.550329,дополнительное образование,0
4,0,гражданский брак,1,158616.077870,сыграть свадьбу,0
5,0,гражданский брак,1,255763.565419,покупка жилья,0
6,0,женат / замужем,0,240525.971920,операции с жильем,0
7,0,женат / замужем,0,135823.934197,образование,0
8,2,гражданский брак,1,95856.832424,на проведение свадьбы,0
9,0,женат / замужем,0,144425.938277,покупка жилья для семьи,0


Переименуем столбец `total_income`, чтобы акцентировать внимание на том, что это **месячный доход  заёмщика**:

* `'total_income'` → `'monthly_income'`

In [19]:
# переименование столбца
df = df.rename(columns={'total_income': 'monthly_income'})

Проверим результат. Для этого ещё раз выведем на экран названия столбцов:

In [20]:
#вывод на экран новых названий столбцов
df.columns

Index(['children', 'family_status', 'family_status_id', 'monthly_income',
       'purpose', 'debt'],
      dtype='object')

Из информации о новой таблице видим, что пропущенные значения есть в столбце `monthly_income`, которые предстоит устранить:

In [21]:
# получение общей информации по обновлённой таблице
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   family_status     21454 non-null  object 
 2   family_status_id  21454 non-null  int64  
 3   monthly_income    19351 non-null  float64
 4   purpose           21454 non-null  object 
 5   debt              21454 non-null  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 1005.8+ KB


Особенность столбца `monthly_income` - все заполненные (ненулевые) ячейки уникальны, 19351 уникальное значение (19352-ое - пропуск):

In [22]:
# подсчет количества уникальных значений в стобце 'monthly_income'
len(df['monthly_income'].unique())

19352

Из проверки уникальных значений столбца с информацией о наличии детей видим, что есть аномальные значения (или в какой-то мере неявные дубликаты) `20` и `-1`:

In [23]:
# подсчет количества уникальных значений в стобце 'childrens'
df['children'].value_counts()

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Проверка уникальных значений столбца о семейном положении показывает 5 категорий клиентов, среди которых не обнаружено неявных дубликатов: 

In [24]:
# подсчет количества уникальных значений в стобце 'family_status'
df['family_status'].value_counts()

женат / замужем          12339
гражданский брак          4151
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

Для семейного положения введён классификатор, в котором также 5 категорий:

In [25]:
# подсчет количества уникальных значений в стобце 'family_status_id'
df['family_status_id'].value_counts()

0    12339
1     4151
4     2810
3     1195
2      959
Name: family_status_id, dtype: int64

Проверим есть ли дубликаты в словаре семейного положения клиентов. Для начала создадим словарь семейного положения клиентов:

In [26]:
# создание классификатора семейного положения клиентов
family_status_dict = df[['family_status_id','family_status']]

# вывод на экран первых 10 строк таблицы
family_status_dict.head(10)

,family_status_id,family_status
0,0,женат / замужем
1,0,женат / замужем
2,0,женат / замужем
3,0,женат / замужем
4,1,гражданский брак
5,1,гражданский брак
6,0,женат / замужем
7,0,женат / замужем
8,1,гражданский брак
9,0,женат / замужем


Для проверки правильности соответствия номера `family_status_id` одному семейному положению воспользуемся методом удаления явных дубликатов. 

Как видно из результата, одному `family_status_id` соответствует  только одно семейное положение, значит классификатор корректен:

In [27]:
#удаление дубликатов для проверки соответствия family_status_id
#одному семейному положению
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)

#вывод на экран
family_status_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


Анализ уникальных значений в столбце `purpose` показывает, что популярнее всего у заёмщиков свадьба, но сразу обнаружен ряд *неявных дубликатов*. С этим столбцом предстоит поработать, будем использовать категоризацию данных:

In [28]:
#подсчет количества уникальных значений в стобце 'purpose'
#и сортировка по убыванию
df['purpose'].value_counts().sort_values(ascending=False)

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

В целевом столбце `debt` дубликатов и пропущенных значений не обнаружено:

In [29]:
#подсчет количества уникальных значений в стобце 'debt'
df['debt'].value_counts()

0    19713
1     1741
Name: debt, dtype: int64

**Вывод**

В основной таблице удалены явные дубликаты после небольшой подготовки столбцов, не имеющих отношения к выдвинутым гипотезам.

Выбраны столбцы из исходной таблицы, информация в которых потребуется для проверки выдвинутых гипотез.

Проверен классификатор семейного положения клиентов, проблем в нём не обнаружено.

Обнаружены следующие проблемы в данных, требующие дальнейшего решения:
- пропущенные значения в столбце `monthly_income`, также в нём неудобный формат данных
- аномалии (или неявные дубликаты `20` и `-1`) в столбце `childrens`
- в столбце `purpose` неявные дубликаты, потребуется категоризация данных для проверки гипотезы

### Обработка пропусков

Как было отмечено в предыдущем разделе пропуски обнаружены в столбце `monthly_income`:

In [30]:
# подсчёт пропусков
df.isna().sum()

children               0
family_status          0
family_status_id       0
monthly_income      2103
purpose                0
debt                   0
dtype: int64

Определим тип пропусков в столбце с использованием логической индексации:

In [31]:
# вывод на экран строк с пустыми ячейками в столбце 'monthly_income'
df[df['monthly_income'].isna()==True].head(10)

,children,family_status,family_status_id,monthly_income,purpose,debt
12,0,гражданский брак,1,NaN,сыграть свадьбу,0
26,0,женат / замужем,0,NaN,образование,0
29,0,Не женат / не замужем,4,NaN,строительство жилой недвижимости,0
41,0,женат / замужем,0,NaN,сделка с подержанным автомобилем,0
55,0,гражданский брак,1,NaN,сыграть свадьбу,1
65,0,Не женат / не замужем,4,NaN,операции с коммерческой недвижимостью,0
67,0,женат / замужем,0,NaN,покупка жилья для семьи,0
72,1,женат / замужем,0,NaN,операции с коммерческой недвижимостью,0
82,2,женат / замужем,0,NaN,жилье,0
83,0,женат / замужем,0,NaN,жилье,0


Видим значение NaN. Причина появления - скорее всего некорректная выгрузка данных, так как при получении кредита заёмщик обязан подтвердить доход. Так как это неполностью пропущенные строчки в таблице, то их желательно сохранить для исследования. Более того, так как мы работаем с информацией о выданных кредитах, то в списке клиентов вероятнее всего люди, имеющие доход. Поэтому правильнее будет заполнить таблицу **медианной зарплатой** по списку. Это решение можно подкрепить количеством клиентов с пропущенным доходом, не имеющих долгов по кредиту:

In [32]:
# количество строк в таблице клиентов с пустой ячейков в 'monthly_income',
# у которых нет долгов по кредиту
len(df[(df['monthly_income'].isna()==True)& (df['debt']==0)]['monthly_income'])

1933

Медианная зарплата клиентов банка:

In [33]:
#медианная месячная зарплата клиентов банка
median_monthly_income = df['monthly_income'].median()

#вывод на экран её значения
round(median_monthly_income,0)

145018.0

Очевидно, что средняя немного выше, поэтому далее будем работать с медианной:

In [34]:
#средняя месячная зарплата клиентов банка
round(df['monthly_income'].mean(),0)

167422.0

Заменим пустые ячейки столбца `monthly_income` медианной зарплатой клиентов с помощью метода `fillna()`:

In [35]:
#заменяем пустые ячейки в столбце 'monthly_income' медианной зарплатой клиентов
df['monthly_income'] = df['monthly_income'].fillna(value=median_monthly_income)

#проверяем изменилась ли медианная зарплата после заполнения ячеек
round(df['monthly_income'].median(),0)

145018.0

Заполнение пустых ячеек медианной зарплатой несколько снизило среднюю, но незначительно, соответственно, не должно ухудшить результаты исследования:

In [36]:
#средняя месячная зарплата клиентов банка
round(df['monthly_income'].mean(),0)

165226.0

Проверим успешность заполнения пропусков:

In [37]:
# подсчёт пропусков
df.isna().sum()

children            0
family_status       0
family_status_id    0
monthly_income      0
purpose             0
debt                0
dtype: int64

**Вывод**

Пропуски в столбце `montly_income` заменены на медианную зарплату клиентов. Решение обосновано, так как большинство клиентов с пропущенным доходом не имеют долгов по кредиту. Далее заменим тип данных в этом столбце для удобства.

### Замена типа данных

Приведём ежемесячный доход клиентов банка к целочисленному формату для удобства отображения. Для этого используем метод `astype`, как наиболее подходящий для этой цели:

In [38]:
#приведём доход клиентов банка к целочисленному формату
df['monthly_income'] = df['monthly_income'].astype('int')

Проверим результат, выведем на экран 10 строк таблицы:

In [39]:
# выведем на экран 10 строк таблицы
df.head(10)

,children,family_status,family_status_id,monthly_income,purpose,debt
0,1,женат / замужем,0,253875,покупка жилья,0
1,1,женат / замужем,0,112080,приобретение автомобиля,0
2,0,женат / замужем,0,145885,покупка жилья,0
3,3,женат / замужем,0,267628,дополнительное образование,0
4,0,гражданский брак,1,158616,сыграть свадьбу,0
5,0,гражданский брак,1,255763,покупка жилья,0
6,0,женат / замужем,0,240525,операции с жильем,0
7,0,женат / замужем,0,135823,образование,0
8,2,гражданский брак,1,95856,на проведение свадьбы,0
9,0,женат / замужем,0,144425,покупка жилья для семьи,0


**Вывод**

Формат столбца доходов клиентов банка изменён на целочисленный формат для удобства.

### Обработка дубликатов

В выбранных для анализа данных обнаружено достаточно большое количество явных дубликатов, больше чем в основной таблице исходных данных `data`:

In [40]:
#явные дубликаты в столбцах, выбранных для проверки гипотез
df[df.duplicated()]['children'].count()

1619

Однако их не следует удалять, так как рассматриваются разные люди, о чем было сказано выше. Явные дубликаты из исходного набора данных удалены ранее. Новые явные дубликаты появились, когда мы исключили из рассмотрения не влияющие на результат проверки гипотез столбцы.

Решим проблему с аномалиями (неявными дубликатами) в столбце `children`:

In [41]:
#подсчет количества уникальных значений в стобце 'children'
df['children'].value_counts()

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Предположим, что `20` и `-1` - это неявные дубликаты значений `2` и `1`. Причина их появления - вероятнее всего - человеческий фактор. `-1` также может быть маркером, обозначающим что-то. 

Заменим с помощью метода `replace` и сразу проверим результат замены.

In [42]:
#заменая неявных дубликатов
df['children']=df['children'].replace(20,2)
df['children']=df['children'].replace(-1,1)

#проверка результата замены
df['children'].value_counts()

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

Также, учитывая долю этих значений (в сумме около 0,56%):

In [43]:
## процент с '20' детьми
print('{:.4%}'.format(76/len(df['children'])))

0.3542%


In [44]:
#процент с '-1' детьми
print('{:.4%}'.format(47/len(df['children'])))

0.2191%


Можно было бы от них избавиться, но в данном случае поступим консервативно.

**Вывод**

Явные дубликаты удалены перед подготовкой исходной таблицы к анализу. Неявные дубликаты устранены в столбце `children`. Дальнейшее устранение дубликатов в столбце `purpose` требует лемматизации и категоризации.

### Лемматизация и категоризация данных

Рассмотрим уникальные значения столбца `purpose`:

In [45]:
#вывод на экран уникальных значений
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Общее количество уникальных значений столбца `purpose`:

In [46]:
#количество уникальных значений целей кредита
len(df['purpose'].unique())

38

Чтобы избежать неявных дубликатов в столбце `purpose` проведём лемматизацию целей и категоризацию по ним. Для этого выделем 4 категории целей кредита:
- покупка автомобиля
- свадьба
- образование
- операции с недвижимостью

Для лемматизации и категоризации целей кредита составим специальную функцию `lemma_group` с использованием библиотеки `pymystem3`:

In [47]:
def lemma_group(purpose):
#    Возвращает категорию целей кредита, используя правила:
#    - если в цели кредита есть лемма 'автомобиль', 
#      то категория "покупка автомобиля";
#    - если в цели кредита есть лемма 'свадьба', 
#      то категория "свадьба";
#    - если в цели кредита есть лемма 'образование', 
#      то категория "образование";
#    - если в цели кредита есть леммы 'жильё' или 'недвижимость', 
#      то категория "операции с недвижимостью";

    
    lemmas = m.lemmatize(purpose)
    
    if ('жилье' in lemmas) or ('недвижимость' in lemmas):
        return 'операции с недвижимостью'
    if 'автомобиль' in lemmas:
        return 'покупка автомобиля'
    if 'свадьба' in lemmas:
        return 'свадьба'
    if 'образование' in lemmas:
        return 'образование'


Проверим результат работы функции. Для этого создадим новый столбец `purpose_group`, сохраним в него результат применения созданной нами функции, выведем на экран уникальные значения категорий:

In [48]:
#создание нового столбца и применение функции
df['purpose_group']=df['purpose'].apply(lemma_group)

#Вывод уникальных значений - новых категорий целей кредита
df['purpose_group'].unique()

array(['операции с недвижимостью', 'покупка автомобиля', 'образование',
       'свадьба'], dtype=object)

Составим словарь соотвествия целей кредита заданной нами категории:

In [49]:
#словарь соответсвия целей кредита заданной категории
purpose_dict = df[['purpose_group','purpose']]

#Вывод на экран первых 10 значений:
purpose_dict.head(10)

,purpose_group,purpose
0,операции с недвижимостью,покупка жилья
1,покупка автомобиля,приобретение автомобиля
2,операции с недвижимостью,покупка жилья
3,образование,дополнительное образование
4,свадьба,сыграть свадьбу
5,операции с недвижимостью,покупка жилья
6,операции с недвижимостью,операции с жильем
7,образование,образование
8,свадьба,на проведение свадьбы
9,операции с недвижимостью,покупка жилья для семьи


Теперь проверим правильность соответствия категории цели кредита. 
Для этого в словаре целей удалим явные дубликаты и обновим нумерацию, удалив старую:

In [50]:
#удаляем дубликаты в словаре и обновляем индексацию, старую удаляем
purpose_dict = purpose_dict.drop_duplicates().reset_index(drop=True)

#отображение словаря на экране
purpose_dict

,purpose_group,purpose
0,операции с недвижимостью,покупка жилья
1,покупка автомобиля,приобретение автомобиля
2,образование,дополнительное образование
3,свадьба,сыграть свадьбу
4,операции с недвижимостью,операции с жильем
5,образование,образование
6,свадьба,на проведение свадьбы
7,операции с недвижимостью,покупка жилья для семьи
8,операции с недвижимостью,покупка недвижимости
9,операции с недвижимостью,покупка коммерческой недвижимости


Прокерка показывает, что функция сработала корректно. Выделено 4 категории целей кредита для проверки одной из гипотез.

**Вывод**

С помощью лематизации выполнена категоризация целей кредита заёмщиков, которая будет использова при проверке одной из гипотез. 

Выделено **4 основные цели кредита** вместо 38 исходных:
- покупка автомобиля, 
- свадьба, 
- образование, 
- операции с недвижимостью.

### Категоризация данных

**Категоризация по наличию детей**

Проведём категоризацию данных по наличию детей у заёмщиков. Для этого ещё раз просмотрим уникальные значения этого столбца:

In [51]:
#просмотр уникальных значений столбца 'children'
df['children'].value_counts()

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

Сделаем функцию для категоризации данных в столбце, в котором содержится инфомрацию о количестве детей у заёмщиков. Так как согласно выдвинутой гипотезе нам нужно проверить влияет ли наличие детей на способность выплачивать кредит, а не их количество, то введём две категории: `есть дети` и `нет детей`. 

Функцию для категоризации запишем следующим образом:

In [52]:
def children_group(children):
    #функция разделяет заёмщиков на две категории:
    #'есть дети' и 'нет детей'
    if children > 0:
        return 'есть дети'
    return 'нет детей'

Проверим результат работы функции. 

Для этого создадим новый столбец `children_group`, сохраним в него результат применения созданной нами функции:

In [53]:
#применение функции и создание нового столбца
df['children_group']=df['children'].apply(children_group)

Выведем на экран уникальные значения:

In [54]:
#Вывод уникальных значений - категории наличия детей
df['children_group'].value_counts()

нет детей    14091
есть дети     7363
Name: children_group, dtype: int64

Создадим словарь, в котором сохраним соответствие количества детей информации о категории по данному признаку:

In [55]:
#словарь соответсвия категорий о наличии детей
children_dict = df[['children_group','children']]

#Вывод на экран первых 10 значений:
children_dict.head(10)

,children_group,children
0,есть дети,1
1,есть дети,1
2,нет детей,0
3,есть дети,3
4,нет детей,0
5,нет детей,0
6,нет детей,0
7,нет детей,0
8,есть дети,2
9,нет детей,0


Проверим соотвествие количества детей заданной нами категории и выведем на экран:

In [56]:
#удаляем дубликаты в словаре и обновляем индексацию, старую удаляем
children_dict = children_dict.drop_duplicates().reset_index(drop=True)

#выводим на экран словарь соответствия категории о наличии детей 
#у заёмщиков количеству детей
children_dict.sort_values('children')

,children_group,children
1,нет детей,0
0,есть дети,1
3,есть дети,2
2,есть дети,3
4,есть дети,4
5,есть дети,5


**Категоризация по уровню дохода**

Для начала подробнее ознакомимся с данными, содержащимися в столбце `monthly_income`. Выведем 15 максимальных значений:

In [57]:
#вывод 15 максимальных значений в столбце 'monthly_income'
df['monthly_income'].sort_values(ascending = False).head(15)

12390    2265604
19548    2200852
9159     1726276
20742    1715018
17137    1711309
17460    1597613
18321    1551152
18306    1427934
15234    1350245
11052    1286280
2224     1278622
7441     1240165
19754    1223042
15177    1172459
13066    1128836
Name: monthly_income, dtype: int64

Некоторые заёмщики имеют ежемесячный доход более 1 млн. руб. Посмотрим сколько всего значений в столбце:

In [58]:
#общее количество значений в столбце 'monthly_income'
df['monthly_income'].count()

21454

Посмотрим максимальное, среднее, медианное и минимальное значение доходов клиентов:

In [59]:
#максимальная зарплата клиентов
df['monthly_income'].max()

2265604

In [60]:
#средняя зарплата клиентов
df['monthly_income'].mean()

165225.6006339144

In [61]:
#медианная зарплата клиентов
df['monthly_income'].median()

145017.0

In [62]:
#минимальная зарплата клиентов
df['monthly_income'].min()

20667

Учитывая, что общее количество заёмщиков около 20 тыс. человек, то разобъём данные о доходах клиентов на 5 категорий. Чтобы категории были показательны для проверки гипотез постараемся выделить 5 диапазонов, в каждой из которых будет не менее 1 тысячи значений.

В результате анализа выявлено, что данные о доходах можно разбить на 5 диапазонов (категорий), для удобства в названиях категорий опущены нули, диапазон доходов приведен в тысячах:
 - `20-75`,
 - `75-145`,
 - `145-200`,
 - `200-300`,
 - `>300`.

Максимальное количество значений приходится на диапазоны `145-200` и `200-300`.
 
 Далее приведём информацию о количестве ячеек в каждом из выбранных диапазонов, чтобы показать, что диапазоны имеют свыше 1000 значений:

In [63]:
#количество значений в диапазоне 20000-75000
df[(df['monthly_income']>20000) &
   (df['monthly_income']<75000)]['monthly_income'].count()

1865

In [64]:
#количество значений в диапазоне 75000-145000
df[(df['monthly_income']>75000) &
   (df['monthly_income']<145000)]['monthly_income'].count()

7809

In [65]:
#количество значений в диапазоне 145000-200000
df[(df['monthly_income']>145000) &
   (df['monthly_income']<200000)]['monthly_income'].count()

6714

In [66]:
#количество значений в диапазоне 200000-300000
df[(df['monthly_income']>200000) &
   (df['monthly_income']<300000)]['monthly_income'].count()

3584

In [67]:
#количество значений в диапазоне свыше 300000
df[(df['monthly_income']>300000)]['monthly_income'].count()

1482

Запишем функцию для категоризации данных о доходах клиентов:

In [68]:
def monthly_income_group(money):
#функция разделяем клиентов на 5 категорий по доходам
# в тысячах рублей: 20-75, 75-145, 145-200, 200-300, >300
    if 20000<money<75000:
        return '20-75'
    if 75000<money<145000:
        return '75-145'
    if 145000<money<200000:
        return '145-200'
    if 200000<money<300000:
        return '200-300'
    if money>300000:
        return '>300'

Создадим новый столбец с категорией по доходам:

In [69]:
#создаём новый столбец и применяем составленную функцию к столбцу 'monthly_income'
df['monthly_income_group']=df['monthly_income'].apply(monthly_income_group)

Просматриваем уникальные значения в новом столбце:

In [70]:
#вывод уникальных значений в столбце 'monthly_income_group'
df['monthly_income_group'].value_counts()

75-145     7809
145-200    6714
200-300    3584
20-75      1865
>300       1482
Name: monthly_income_group, dtype: int64

Составим словарь соответствия доходов клиентов одной из выбранных категорий (диапазонов), он пригодится в дальнейшем:

In [71]:
#словарь соответсвия категорий информации о доходах
monthly_income_dict = df[['monthly_income_group','monthly_income']]

#вывод на экран первых 10 строк
monthly_income_dict.head(10)

,monthly_income_group,monthly_income
0,200-300,253875
1,75-145,112080
2,145-200,145885
3,200-300,267628
4,145-200,158616
5,200-300,255763
6,200-300,240525
7,75-145,135823
8,75-145,95856
9,75-145,144425


**Вывод**

Выполнена категоризация клиентов по наличию детей и по доходу. Клиенты по количеству детей разделены на категории `есть дети` и `нет детей`. Отдельно можно проверить влияние количества детей, для этого не требуется разделение на каотегории, так как в исходной таблице количество детей уже задано для каждого из клиентов.

Доходы клиентов разделены на 5 категорий, диапазоны выбраны так, чтобы в каждом было более 1000 значений. 

Диапазоны в тысячах рублей: `20-75`, `75-145`, `145-200`, `200-300`, `>300`.

Данные готовы к проверке гипотез.

## Шаг 3. Проверка гипотез

Для проверки гипотез будем рассчитывать конверсию: какой процент заёмщиков от общего числа клиентов имеет задолженность по кредиту **в каждой из категорий**? 

Далее для каждой из гипотез описаны шаги для вычисления указанной конверсии.

### Наличие детей

**- Есть ли зависимость между наличием детей и возвратом кредита в срок?**

С помощью метода `groupby()` выведем общее количество заёмщиков в каждой категории наличия детей, сумму должников и конверсию:

In [72]:
df[['children_group','debt']].groupby('children_group').agg(['count','sum','mean'])

debt                
                count   sum      mean
children_group                       
есть дети        7363   678  0.092082
нет детей       14091  1063  0.075438

Проверим как влияет **количество детей** на способность вернуть кредит в срок.

Проделаем аналогичные шаги для вычисления конверсии, что сделали ранее и выведем на экран сводную таблицу, в которой проследим влияние количества детей на способность клиентов выплатить кредит без задолженности:

In [73]:
#добавление нового столбца в таблицу - информация о количестве клиентов, 
#имеющих задолженность по кредиту, в каждой из категорий
(
    df[['children','debt']]
    .groupby('children')
    .agg(['count','sum','mean'])
    .sort_values(by=('debt','mean'),
                 ascending=False)
)

debt                
          count   sum      mean
children                       
4            41     4  0.097561
2          2128   202  0.094925
1          4855   445  0.091658
3           330    27  0.081818
0         14091  1063  0.075438
5             9     0  0.000000

Подтвердился результат, полученный для двух категорий `есть дети` и `нет детей`. Действительно, клиенты, не имеющие детей чаще выплачивают кредит без долгов. Одно исключение - клиенты, имеющие 5 детей в 100% случаев не имеют задолженности по кредиту, однако слишком маленькая выборка, чтобы принимать данное утверждение.

**Вывод**

*Выдвенутая гипотеза*: есть ли зависимость между наличием детей и возвратом кредита в срок? 

**9,21%** заёмщиков с детьми имеют долги по кредиту, в то время как среди заёмщиков без детей этот показатель равняется **7,54%**. 

Можно ответить утвердительно: да, зависимость есть. Среди заёмщиков, имеющих детей, случаев задолженности по кредиту на **1,67%** больше.

Количество детей также влияет на способность клиента выплатить кредит. Но с возрастанием количества детей, нет равномерного роста неспособности клиента выплатить кредит. Так задолженность имеют (процент клиентов от общего количества заёмщиков данной категории):
- нет детей - 7,54%,
- 1 ребёнок - 9,17%,
- 2 детей - 9,49%,
- 3 детей - 8,18%,
- 4 детей - 9,76%,
- 5 детей - 0%.

Иными словами, с количеством детей 1,2 задолженность имеют 9,17-9,49% заёмщиков, но у клиентов с 3 детьми задолженность имеют 8,18%, затем снова рост - 4 детей и 9,76% клиентов имеют задолженность.

Для модели скоринга вероятнее удобнее пользоваться общей категорией `есть дети` и `нет детей`, чтобы снизить ошибки.

### Семейное положение

 - **Есть ли зависимость между семейным положением и возвратом кредита в срок?**

Общее количество заёмщиков, должников и конверсия в каждой категории семейного положения:

In [74]:
(
    df[['family_status_id','family_status','debt']]
    .groupby(['family_status_id','family_status'])
    .agg(['count','sum','mean'])
)

debt               
                                        count  sum      mean
family_status_id family_status                              
0                женат / замужем        12339  931  0.075452
1                гражданский брак        4151  388  0.093471
2                вдовец / вдова           959   63  0.065693
3                в разводе               1195   85  0.071130
4                Не женат / не замужем   2810  274  0.097509

**Вывод**

В результате проведённого исследования можно сделать вывод, что семейное положение клиентов оказывает некоторое влияние на возврат кредита в срок:
- Наибольшый процент задолженностей у заёмщиков со статусом `не женат/не замужем` и `гражданский брак` - **9,75%** и **9,35%**, соответственно. 
- Примерно одинаковый результат у категорий `женат/замужем` и `в разводе` - **7,55%** и **7,11%**, соответственно.
- Наименьший процент заёмщиков, имеющих задолженность по кредиту, у категории `вдовец/вдова` - **6,57%**

### Уровень дохода

**- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

Общее количество заёмщиков, должников и конверсия:

In [75]:
(
    df[['monthly_income_group','debt']]
    .groupby('monthly_income_group')
    .agg(['count','sum','mean'])
    .sort_values(by=('debt','mean'),
                 ascending=False)
)

debt               
                     count  sum      mean
monthly_income_group                     
145-200               6714  579  0.086238
75-145                7809  668  0.085542
20-75                 1865  136  0.072922
>300                  1482  106  0.071525
200-300               3584  252  0.070312

**Вывод**

В результате проверки гипотезы получено, что примерно одинаковый процент клиентов, имеющих задолженность, наблюдается в категориях доходов `20-75`, `200-300` и `>300`: **7,29%**, **7,03%** и **7,15%**, соответственно.

Немного выше процент клиентов, имеющих задолженность по кредиту, в категориях `75-145` и `145-200`: **8,55%** и **8,62%**

### Цели кредита

**- Как разные цели кредита влияют на его возврат в срок?**

Общее количество заёмщиков, должников и конверсия:

In [76]:
(
    df[['purpose_group','debt']]
    .groupby('purpose_group')
    .agg(['count','sum','mean'])
    .sort_values(by=('debt','mean'),
                 ascending=False)
)

debt               
                          count  sum      mean
purpose_group                                 
покупка автомобиля         4306  403  0.093590
образование                4013  370  0.092200
свадьба                    2324  186  0.080034
операции с недвижимостью  10811  782  0.072334

**Вывод**

В результате проведенного исследования можно сделать вывод, что цель кредита оказывает некоторое влияние на возврат кредита в срок:
- **7,23%** задолженностей в категории "операции с недвижимостью",
- **8%** в категории "свадьба",
- **9,22%** и **9,35%** в категориях образование и покупка автомобиля, соответственно.

## Шаг 4. Общий вывод

В результате исследования было проверено 4 гипотезы и сделаны соответсвующие выводы.

**1. Есть ли зависимость между наличием детей и возвратом кредита в срок?**

**9,21%** заёмщиков с детьми имеют долги по кредиту, в то время как среди заёмщиков без детей этот показатель равняется **7,54%**. 

Можно ответить утвердительно: да, зависимость есть. Среди заёмщиков, имеющих детей, случаев задолженности по кредиту на **1,67%** больше.

Количество детей также влияет на способность клиента выплатить кредит. Но с возрастанием количества детей, нет равномерного роста неспособности клиента выплатить кредит: с количеством детей `1` и `2` задолженность имеют 9,17-9,49% заёмщиков, но у клиентов с `3` детьми задолженность имеют 8,18%, затем снова рост - 4 детей и 9,76% клиентов имеют задолженность.

Для модели скоринга вероятнее удобнее пользоваться обобщёнными категориями `есть дети` и `нет детей`.

**2. Есть ли зависимость между семейным положением и возвратом кредита в срок?**

Cемейное положение клиентов оказывает влияние на возврат кредита в срок:
- Наибольшый процент задолженностей у заёмщиков со статусом `не женат/не замужем` и `гражданский брак` - **9,75%** и **9,35%**, соответственно. 
- Примено одинаковый результат у категорий `женат/замужем` и `в разводе` - **7,55%** и **7,11%**, соответственно.
- Наименьший процент заёмщиков, имеющих задолженность по кредиту, у категории `вдовец/вдова` - **6,57%**

**3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

В результате проверки гипотезы получено, что примерно одинаковый процент клиентов, имеющих задолженность, наблюдается в категориях доходов (в тысячах рублей) `20-75`, `200-300` и `>300`: **7,29%**, **7,03%** и **7,15%**, соответственно.

Немного выше процент клиентов, имеющих задолженность по кредиту, в категориях `75-145` и `145-200`: **8,55%** и **8,62%**

Можно сделать вывод, что гипотеза скорее не подтвердилась. Нельзя сказать, что чем выше доход, тем выше вероятность выплатить кредит без долгов. Задолженность по кредиту имеют как клиенты с высоким уровнем дохода, так и с низким. Более высокий процент в категориях `75-145` и `145-200`.

**4. Как разные цели кредита влияют на его возврат в срок?**

Цель кредита оказывает некоторое влияние на возврат кредита в срок:
- **7,23%** задолженностей в категории "операции с недвижимостью",
- **8%** в категории "свадьба",
- **9,22%** и **9,35%** в категориях образование и покупка автомобиля, соответственно.
